# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [94]:
import pandas 
import sqlite3
import warnings
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [95]:
conn = sqlite3.connect('sql/go_sales_train.sqlite')

print("SQLite3 Connection: ",conn)

SQLite3 Connection:  <sqlite3.Connection object at 0x11b915d50>


<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [96]:
import os

db_path = os.path.abspath("sql/go_sales_train.sqlite")

sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
conn = sqlite3.connect(db_path)

#Vul dit codeblok verder in
cursor = conn.cursor()
cursor.execute(sql_query)

tables = cursor.fetchall()


print('De tabellen uit de database: ')
for table in tables:
    print(table[0])



De tabellen uit de database: 
country
order_details
order_header
order_method
product
product_line
product_type
retailer_site
return_reason
returned_item
sales_branch
sales_staff


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [97]:

dataframe = pandas.read_sql_query('SELECT * FROM sales_staff',conn)

print(dataframe.head())

   SALES_STAFF_CODE FIRST_NAME  LAST_NAME                      POSITION  \
0                 4      Denis       Pagé               General Manager   
1                 5  Élizabeth     Michel  Level 3 Sales Representative   
2                 6      Émile   Clermont  Level 1 Sales Representative   
3                 7    Étienne     Jauvin  Level 2 Sales Representative   
4                12    Elsbeth  Wiesinger  Level 2 Sales Representative   

          WORK_PHONE  EXTENSION                FAX                   EMAIL  \
0  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60       DPage@grtd123.com   
1  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60     EMichel@grtd123.com   
2  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   EClermont@grtd123.com   
3  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60     EJauvin@grtd123.com   
4  +(49) 40 663 1990     1819.0  +(49) 40 663 4571  EWiesinger@grtd123.com   

   DATE_HIRED  SALES_BRANCH_CODE          FULL_NAME  
0  1996-11-03             

## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [98]:
sales_staff = pandas.read_sql('SELECT * FROM sales_staff',conn)
sales_staff['DATE_HIRED'] = pandas.to_datetime(sales_staff['DATE_HIRED'])
print(sales_staff.dtypes)


SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION                     object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
FULL_NAME                    object
dtype: object


Als we hier het jaar uit willen halen, schrijven we:

In [99]:
pandas.DatetimeIndex(sales_staff['DATE_HIRED']).year


Index([1996, 1995, 1998, 1997, 1997, 1999, 1997, 1998, 1996, 1996,
       ...
       1999, 2000, 1999, 1999, 1999, 1999, 1998, 1998, 2002, 2002],
      dtype='int32', name='DATE_HIRED', length=104)

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [100]:
sales_staff_code = sales_staff['SALES_STAFF_CODE']
work_phone = sales_staff['WORK_PHONE']
extension = sales_staff['EXTENSION']
fax = sales_staff['FAX']
email= sales_staff['EMAIL']

print(sales_staff_code)


0        4
1        5
2        6
3        7
4       12
      ... 
99     122
100    123
101    124
102    420
103    420
Name: SALES_STAFF_CODE, Length: 104, dtype: int64


Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [101]:
contact_details = pandas.concat([sales_staff_code,work_phone,extension,fax,email],axis=1)

print(contact_details)

     SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                   4  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60   
1                   5  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60   
2                   6  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   
3                   7  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60   
4                  12  +(49) 40 663 1990     1819.0  +(49) 40 663 4571   
..                ...                ...        ...                ...   
99                122    +32 16 20.73.21     1634.0    +32 16 20.73.32   
100               123  +(43) 13 79 56 32      326.0  +(43) 13 79 56 33   
101               124  +(43) 13 79 56 32      349.0  +(43) 13 79 56 33   
102               420          316123456     1993.0          316123456   
103               420          316123456     1993.0          316123456   

                      EMAIL  
0         DPage@grtd123.com  
1       EMichel@grtd123.com  
2     EClermont@grtd1

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [102]:
contact_details_nieuw = contact_details.head(5)

print(contact_details_nieuw)

   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1819.0  +(49) 40 663 4571   

                    EMAIL  
0       DPage@grtd123.com  
1     EMichel@grtd123.com  
2   EClermont@grtd123.com  
3     EJauvin@grtd123.com  
4  EWiesinger@grtd123.com  


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [103]:
first_language = ['EN','EN','EN','EN','EN']
first_language_df = pandas.DataFrame({'FIRST_LANGUAGE' : first_language})

contact_details_nieuw= pandas.concat([sales_staff_code,work_phone,extension,fax,email,first_language_df],axis=1)

print(contact_details_nieuw.head(5))


   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1819.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE  
0       DPage@grtd123.com             EN  
1     EMichel@grtd123.com             EN  
2   EClermont@grtd123.com             EN  
3     EJauvin@grtd123.com             EN  
4  EWiesinger@grtd123.com             EN  


Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [104]:
second_languages = {
    0: 'FR',
    1: 'FR',
    2: 'FR',
    3: 'DE',
    4: 'DE'
}
second_languages_df = pandas.DataFrame.from_dict(second_languages, orient='index',columns=["SECOND_LANGUAGE"])

contact_details_nieuw= pandas.concat([sales_staff_code,work_phone,extension,fax,email,first_language_df,second_languages_df],axis=1)
print(contact_details_nieuw.head(5))



   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1819.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE SECOND_LANGUAGE  
0       DPage@grtd123.com             EN              FR  
1     EMichel@grtd123.com             EN              FR  
2   EClermont@grtd123.com             EN              FR  
3     EJauvin@grtd123.com             EN              DE  
4  EWiesinger@grtd123.com             EN              DE  


Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [105]:
third_language = {
    'THIRD_LANGUAGE': ['NL','NL','JPN','JPN','KOR']
    
}
third_language_df = pandas.DataFrame(third_language)

contact_details_nieuw= pandas.concat([sales_staff_code,work_phone,extension,fax,email,first_language_df,second_languages_df,third_language_df],axis=1)
print(contact_details_nieuw.head(5))

#Ik hoef de nieuw ontstande kolom geen naam meer te geven omdat de de key van de dict, de naam wordt denk ik


   SALES_STAFF_CODE         WORK_PHONE  EXTENSION                FAX  \
0                 4  +33 1 68 94 52 20      326.0  +33 1 68 94 56 60   
1                 5  +33 1 68 94 52 20      337.0  +33 1 68 94 56 60   
2                 6  +33 1 68 94 52 20      379.0  +33 1 68 94 56 60   
3                 7  +33 1 68 94 52 20      399.0  +33 1 68 94 56 60   
4                12  +(49) 40 663 1990     1819.0  +(49) 40 663 4571   

                    EMAIL FIRST_LANGUAGE SECOND_LANGUAGE THIRD_LANGUAGE  
0       DPage@grtd123.com             EN              FR             NL  
1     EMichel@grtd123.com             EN              FR             NL  
2   EClermont@grtd123.com             EN              FR            JPN  
3     EJauvin@grtd123.com             EN              DE            JPN  
4  EWiesinger@grtd123.com             EN              DE            KOR  


## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [106]:

nieuwe_rij = (420,'Sam','Stille','CEO',316123456,1992, 316123456,'samstille2712Yahoo.nl','2002-12-27',1)

cursor.execute('INSERT INTO sales_staff (SALES_STAFF_CODE, FIRST_NAME, LAST_NAME, POSITION_EN, WORK_PHONE, EXTENSION, FAX, EMAIL, DATE_HIRED, SALES_BRANCH_CODE) VALUES (?,?,?,?,?,?,?,?,?,?)',nieuwe_rij)
conn.commit()

cursor.execute('SELECT * FROM sales_staff WHERE SALES_STAFF_CODE = ?',(420,))

rij= cursor.fetchone()

print(rij)



OperationalError: table sales_staff has no column named POSITION_EN

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [53]:
#cursor.execute("ALTER TABLE sales_staff ADD COLUMN FULL_NAME TEXT ")
conn.commit()
cursor.execute("UPDATE sales_staff SET FULL_NAME = FIRST_NAME || ' ' || LAST_NAME WHERE FIRST_NAME IS NOT NULL AND LAST_NAME IS NOT NULL")
conn.commit()

## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [54]:
print(sales_staff.dtypes)

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION_EN                  object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
FULL_NAME                    object
dtype: object


Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [107]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

SALES_STAFF_CODE              int64
FIRST_NAME                   object
LAST_NAME                    object
POSITION                     object
WORK_PHONE                   object
EXTENSION                   float64
FAX                          object
EMAIL                        object
DATE_HIRED           datetime64[ns]
SALES_BRANCH_CODE             int64
FULL_NAME                    object
dtype: object

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [108]:
cursor.execute("UPDATE sales_staff SET EXTENSION = EXTENSION + 1")

conn.commit()

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [109]:
cursor.execute("UPDATE sales_staff SET POSITION_EN = 'General Manager' WHERE POSITION_EN = 'Branch Manager' ")

conn.commit()

OperationalError: no such column: POSITION_EN

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [110]:
cursor.execute("ALTER TABLE sales_staff RENAME COLUMN POSITION_EN TO POSITION")
conn.commit()

OperationalError: no such column: "POSITION_EN"

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [111]:
#sales_staff = sales_staff.drop([99,100,101])

conn.commit()

print(sales_staff.index)


RangeIndex(start=0, stop=104, step=1)


### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [112]:
#sales_staff = sales_staff.drop('FAX',axis=1)

print(sales_staff)

     SALES_STAFF_CODE FIRST_NAME  LAST_NAME                      POSITION  \
0                   4      Denis       Pagé               General Manager   
1                   5  Élizabeth     Michel  Level 3 Sales Representative   
2                   6      Émile   Clermont  Level 1 Sales Representative   
3                   7    Étienne     Jauvin  Level 2 Sales Representative   
4                  12    Elsbeth  Wiesinger  Level 2 Sales Representative   
..                ...        ...        ...                           ...   
99                122     Yvette    Lattrez  Level 3 Sales Representative   
100               123      Willi  Seefelder  Level 2 Sales Representative   
101               124     Sabine     Grüner  Level 3 Sales Representative   
102               420        Sam     Stille                           CEO   
103               420        Sam     Stille                           CEO   

            WORK_PHONE  EXTENSION                FAX                   EMAI